In [51]:
import os,re,json,csv
bible_folder = "/Users/amycweng/DH/Early-Modern-Sermons/assets/bible"

standard_books = {
    'GEN': "Genesis", 'EXO': "Exodus", 'LEV': "Leviticus", 
    'NUM': "Numbers", 'DEU': "Deuteronomy", 'JOS': "Joshua", 
    'JDG': "Judges", 'RUT': "Ruth", '1SA': "1 Samuel", 
    '2SA': "2 Samuel", '1KI': "1 Kings", '2KI': "2 Kings", 
    '1CH': "1 Chronicles", '2CH': "2 Chronicles", 'EZR': "Ezra", 
    'NEH': "Nehemiah", 'EST': "Esther", 'JOB': "Job", 
    'PSA': "Psalms", 'PRO': "Proverbs", 'ECC': "Ecclesiastes", 
    'SNG': "Canticles", 'ISA': "Isaiah", 'JER': "Jeremiah", 
    'LAM': "Lamentations", 'EZK': "Ezekiel", 'DAN': "Daniel", 
    'HOS': "Hosea", 'JOL': "Joel", 'AMO': "Amos", 
    'OBA': "Obadiah", 'JON': "Jonah", 'MIC': "Micah", 
    'NAM': "Nahum", 'HAB': "Habakkuk", 'ZEP': "Zephaniah", 
    'HAG': "Haggai", 'ZEC': "Zechariah", 'MAL': "Malachi", 
    'MAT': "Matthew", 'MRK': "Mark", 'LUK': "Luke", 
    'JHN': "John", 'ACT': "Acts", 'ROM': "Romans", 
    '1CO': "1 Corinthians", '2CO': "2 Corinthians", 
    'GAL': "Galatians", 'EPH': "Ephesians", 'PHP': "Philippians", 
    'COL': "Colossians", '1TH': "1 Thessalonians", 
    '2TH': "2 Thessalonians", '1TI': "1 Timothy", 
    '2TI': "2 Timothy", 'TIT': "Titus", 'PHM': "Philemon", 
    'HEB': "Hebrews", 'JAS': "James", '1PE': "1 Peter", 
    '2PE': "2 Peter", '1JN': "1 John", '2JN': "2 John", 
    '3JN': "3 John", 'JUD': "Jude", 'REV': "Revelation", 
    'TOB': "Tobit", 'JDT': "Judith", 'WIS': "Wisdom", 
    'SIR': "Ecclesiasticus", 'BAR': "Baruch", '1MA': "1 Maccabees", 
    '2MA': "2 Maccabees", 'ESG': "Esther", 
    'S3Y': "Song of the Three Young Men", 'SUS': "Susanna", 
    'BEL': "Bel and the Dragon", '1ES': "1 Esdras", 
    'MAN': "Manasseh", '2ES': "2 Esdras"
}

book_parts = {
    'GEN': "Old Testament", 'EXO': "Old Testament", 'LEV': "Old Testament", 
    'NUM': "Old Testament", 'DEU': "Old Testament", 'JOS': "Old Testament", 
    'JDG': "Old Testament", 'RUT': "Old Testament", '1SA': "Old Testament", 
    '2SA': "Old Testament", '1KI': "Old Testament", '2KI': "Old Testament", 
    '1CH': "Old Testament", '2CH': "Old Testament", 'EZR': "Old Testament", 
    'NEH': "Old Testament", 'EST': "Old Testament", 'JOB': "Old Testament", 
    'PSA': "Old Testament", 'PRO': "Old Testament", 'ECC': "Old Testament", 
    'SNG': "Old Testament", 'ISA': "Old Testament", 'JER': "Old Testament", 
    'LAM': "Old Testament", 'EZK': "Old Testament", 'DAN': "Old Testament", 
    'HOS': "Old Testament", 'JOL': "Old Testament", 'AMO': "Old Testament", 
    'OBA': "Old Testament", 'JON': "Old Testament", 'MIC': "Old Testament", 
    'NAM': "Old Testament", 'HAB': "Old Testament", 'ZEP': "Old Testament", 
    'HAG': "Old Testament", 'ZEC': "Old Testament", 'MAL': "Old Testament", 

    'MAT': "New Testament", 'MRK': "New Testament", 'LUK': "New Testament", 
    'JHN': "New Testament", 'ACT': "New Testament", 'ROM': "New Testament", 
    '1CO': "New Testament", '2CO': "New Testament", 
    'GAL': "New Testament", 'EPH': "New Testament", 'PHP': "New Testament", 
    'COL': "New Testament", '1TH': "New Testament", '2TH': "New Testament", 
    '1TI': "New Testament", '2TI': "New Testament", 'TIT': "New Testament", 
    'PHM': "New Testament", 'HEB': "New Testament", 'JAS': "New Testament", 
    '1PE': "New Testament", '2PE': "New Testament", '1JN': "New Testament", 
    '2JN': "New Testament", '3JN': "New Testament", 'JUD': "New Testament", 
    'REV': "New Testament", 

    'TOB': "Apocrypha", 'JDT': "Apocrypha", 'WIS': "Apocrypha", 
    'SIR': "Apocrypha", 'BAR': "Apocrypha", '1MA': "Apocrypha", 
    '2MA': "Apocrypha", 'ESG': "Apocrypha", 'S3Y': "Apocrypha", 
    'SUS': "Apocrypha", 'BEL': "Apocrypha", '1ES': "Apocrypha", 
    'MAN': "Apocrypha", '2ES': "Apocrypha"
}

version_dict = {"eng-kjv_readaloud":"KJV",
           "engDRA_readaloud":"Douay-Rheims",
           "enggnv_readaloud":"Geneva",
           "engtnt_readaloud":"Tyndale",
           "engWycliffe_readaloud":"Wycliffe",
           "latVUC_readaloud":"Vulgate"}

# books = {}
outputfolder = "/Users/amycweng/DH/SERMONS_APP/db/data"

for bname in os.listdir(f"{bible_folder}/raw"): 
    bible = []
    bible_dict = {}
    for fp in sorted(os.listdir(f"{bible_folder}/raw/{bname}")): 
        if ".txt" not in fp or "000_000_000" in fp or ".asc" in fp: continue
        with open(f"{bible_folder}/raw/{bname}/{fp}","r") as file: 
            lines = file.readlines()
        book = fp.split("_")[2]
        part = book_parts[book]
        book = standard_books[book]
        version = version_dict[bname]
        chap = str(int(fp.split("_")[3]))
        # books[fp.split("_")[2]] = None 
        for idx, l in enumerate(lines): 
            if idx < 2: continue
            verse = idx-1 
            l = l.strip("\n ¶")
            key = f"{book} {chap}.{verse} ({version})"
            bible.append({
                "id": key,
                "version": version,
                "part": part, 
                "book":book,
                "chapter":chap,
                "verse":str(verse),
                "text":l
            })
            bible_dict[f"{book} {chap}.{verse}"] = l
    with open(f"{outputfolder}/{version}.csv","w+") as outfile: 
        writer = csv.DictWriter(outfile, fieldnames=bible[0].keys())
        writer.writerows(bible)
    with open(f'{bible_folder}/plain/{version}.json',"w+") as file: 
        json.dump(bible_dict,file)
# print(books)

In [ ]:
import subprocess
repo = '/Users/amycweng/DH/Early-Modern-Sermons/assets' 
for fp in os.listdir(f'{repo}/bible/plain'):
    os.chdir('/Users/amycweng/DH/morphadorner-2')
    print(fp)
    subprocess.run(['./adornplainemetext', f"{repo}/bible/adorned", f"{repo}/bible/plain/{fp}"])

2024-08-01 15:12:58,623 INFO  -    Adornments written to /Users/amycweng/DH/Early-Modern-Sermons/assets/bible/adorned/engDRA_readaloud-001.txt in 70 seconds.
2024-08-01 15:12:58,623 INFO  - All files adorned in 85 seconds.


# Douay-Rheims Online Website of 1582 NT and parts of 1610 OT 

In [ ]:
webpages = {
    'matthew':'mat',
    'mark':'mark',
    'luke':'luke',
    'john':'john',
    'acts':'acts',
    'romans':'romans',
    'i%20corinthians':'i%20corinth',
    'ii%20corinthians':'ii%20corinth',
    'galatians':'galatians',
    'ephesians':'ephesians',
    'philippians':'philippians',
    'colossians':'colossians',
    'i%20thessalonians':"i%20thessalonians",
    'ii%20thessalonians':'ii%20thessalonians',
    'i%20timothee':'i%20timothee',
    'ii%20timothee':'ii%20timothee',
    'titus':'titus',
    'philemon':'philemon',
    'hebrews':'hebrews',
    'james':'james',
    'i%20peter':'i%20peter',
    'ii%20peter':'ii%20peter',
    'i%20john':'i%20john',
    'ii%20john':'ii%20john',
    'iii%20john':'iii%20john',
    'jude':'jude',
    'revelations':'revelations',
    'old/genesis':'old/genesis', # old/ indicates OT 
    'old/exodus':'old/exodus',
    'old/psalms':'old/psalms',
    'old/ruth':'old/ruth',
    'old/wisdom':'old/wisdom',
    'old/lamentations':'old/lamentations',
    'old/baruch':'old/baruch',
    'old/daniel':'old/daniel',
    'old/jonas':'old/jonas',
    'old/sophonias':'old/sophonias'
}

standardize = {
    'revelations':'Revelation',
    'i%20corinthians':'1 Corinthians',
    'ii%20corinthians':'2 Corinthians',
    'i%20thessalonians':"1 Thessalonians",
    'ii%20thessalonians':'2 Thessalonians',
    'i%20timothee':'1 Timothy',
    'ii%20timothee':'2 Timothy',
    'i%20peter':'1 Peter',
    'ii%20peter':'2 Peter',
    'i%20john':'1 John',
    'ii%20john':'2 John',
    'iii%20john':'3 John',
    'old/jonas':'Jonah',
    'sophonias':'Zephaniah',
}


In [ ]:
from bs4 import BeautifulSoup
import requests 

class Bible():
    def __init__(self,target=None):
        self.target = target # either None or (id,name)
        self.website = 'https://originaldouayrheims.com'
        self.level = 0 # category level; 0 level ones are the main root categories 
        self.edges = {self.level:[]} # level: (parent_id, child_id)
        self.id_to_name = {} 


        if target is None: # get the names of all the main categories (41 in total) 
            self.n = 4 # <name> <id> <name> <url> format for the home page 
            result = requests.get(self.website)
            content = result.text
            soup = BeautifulSoup(content, 'lxml')
            self.get_info(soup)
        elif len(target) == 2: # start from a child node 
            self.n = 3 # <name> <id> <url> format for all other subpages
            result = requests.get(self.get_url(self.target))
            if not re.search(r"^\d+$",target[0]):
                print("Error: please enter the category id as the first element of target.")
            else: 
                self.id_to_name[target[0]] = target[1]
                self.edges[0].append((None,self.target[0])) 
        else: 
            print("Error: please enter target in the right format.")
        # print(self.edges, self.id_to_name)

    def num_children(self):
        num = 0 
        for p,c in self.edges[self.level]:
            if c is not None: 
                num += 1 
        return num

    def get_url(self, category_id):
        return f"{self.website}/browseNodeLookup/{category_id}.html"

    def get_info(self,soup,parent_id=None):
        newlevel = self.level + 1 
        table = soup.find_all("td") # entries of the data table on the website page 
        if len(table) == 0: # no data 
            self.edges[newlevel].append((parent_id,None))
        elif "is a leaf node" in table[0].text: # reached a leaf node 
            self.edges[newlevel].append((parent_id,None))
        else:     # there exist children 
            curr_id, curr_name = None, None
            for idx, entry in enumerate(table): 
                t = entry.text
                t = re.sub(r"\s{2,}","",t)
                if (idx+1) % self.n == 1: 
                    curr_name = t
                elif (idx+1) % self.n == 2: 
                    curr_id = t 
                    if curr_id not in self.id_to_name:
                        self.id_to_name[curr_id] = curr_name
                    if parent_id is None: 
                        self.edges[0].append((None,curr_id))         
                    else: 
                        self.edges[newlevel].append((parent_id,curr_id))
                    curr_id, curr_name = None, None
    
    def process_descendents(self,filename):
        self.n = 3 
        while self.num_children() > 0:
            self.edges[self.level+1] = [] 
            for p,curr in self.edges[self.level]: 
                if curr is None: continue 
                link = self.get_url(curr)
                result = requests.get(link)
                content = result.text
                soup = BeautifulSoup(content, 'lxml')
                self.get_info(soup,curr)
                print(f"Processed level {self.level+1} children of {self.id_to_name[curr]} ({curr})")
                with open(f"../categories/{filename}","w+") as file: 
                    json.dump((self.edges,self.id_to_name),file)
            self.level += 1 

def process(parent_id, parent_name):
    categories = Bible((parent_id,parent_name))
    categories.process_descendents(f"{parent_name}.json")
    print(categories.edges, "\n", categories.id_to_name)